### Working with Zillow Dataset
#### <u>``Artifact: IPYNB "Data Wrangle" File``</u>
    
    date: Monday, July 18th 2022
----

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# importing needed libraries/modules
import pandas as pd
import numpy as np

# importing visualization libraries 
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 5)

# importing data wrangle.py file for quicker "importing" of the zillow dataset
from wrangle import get_zillow_dataset, clean_zillow_dataset

# importing sql 
import env
from env import user, password, host, get_connection

----
**``Exercises II``**

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. 

You thought it might look impressive to **build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features**; who knows, you might even do some feature engineering to blow them away. Your **goal is to predict the values of single unit properties using the obervations from 2017.**

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

``Acquire:`` 
1. bedroomcnt
2. bathroomcnt
3. calculatedfinishedsquarefeet
4. taxvaluedollarcnt
5. yearbuilt
6. taxamount, and 
7. fips 

from the zillow database for all **'Single Family Residential'** properties

* Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. 

* You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

* Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.

----

In [3]:
# let's import the needed zillow "properties_2017" table 
# note: incase of any future importing/MySQL issues, i'll also want to save/cache a local .csv file once the data has been initially imported

# query necessary to pull the 2017 properties table from MySQL
# query = ''' 
# SELECT
#         bedroomcnt as bedroom_count,
#         bathroomcnt as bath_count,
#         calculatedfinishedsquarefeet as finished_sq_feet,
#         taxvaluedollarcnt as tax_dollar_value,
#         yearbuilt as year_built,
#         taxamount,
#         fips
# FROM properties_2017
# JOIN propertylandusetype using (propertylandusetypeid)
# WHERE propertylandusedesc = "Single Family Residential"
# '''

# creating the MySQL url with necessary information 
# url = get_connection(user, password, host, "zillow")

# creating the zillow dataframe using Pandas' read_sql() function
# zillow_df = pd.read_sql(query, url)

In [4]:
# creating a zillow.csv file for quicker/cache refencing 
# zillow_df.to_csv("/Users/mijailmariano/codeup-data-science/regression-exercises/zillow.csv", index = True)
# created!

In [5]:
# can now call the zillow dataset from my "wrangle.py" file where the zillow data will be cached for quicker future referencing 

zillow_df = get_zillow_dataset()

In [6]:
# let's confirm the MySQL table import
zillow_df.head()

,bedroom_count,bath_count,finished_sq_feet,tax_dollar_value,year_built,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [7]:
# initial dataframe shape
# 2,152,863 observations/rows
# 7 features/columns

initial_shape = zillow_df.shape
initial_shape

(2152863, 7)

In [8]:
# let's do some further data exploration/familiarization
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   bedroom_count     float64
 1   bath_count        float64
 2   finished_sq_feet  float64
 3   tax_dollar_value  float64
 4   year_built        float64
 5   taxamount         float64
 6   fips              float64
dtypes: float64(7)
memory usage: 131.4 MB


In [9]:
# let's do a .describe() to further understand the features and values
summary_stats = zillow_df.describe().T.round()
summary_stats

,count,mean,std,min,25%,50%,75%,max
bedroom_count,2152852.0,3.0,1.0,0.0,3.0,3.0,4.0,25.0
bath_count,2152852.0,2.0,1.0,0.0,2.0,2.0,3.0,32.0
finished_sq_feet,2144379.0,1863.0,1222.0,1.0,1257.0,1623.0,2208.0,952576.0
tax_dollar_value,2152370.0,461896.0,699676.0,1.0,188170.0,327671.0,534527.0,98428909.0
year_built,2143526.0,1961.0,22.0,1801.0,1949.0,1958.0,1976.0,2016.0
taxamount,2148421.0,5635.0,8179.0,2.0,2535.0,4109.0,6414.0,1337756.0
fips,2152863.0,6048.0,20.0,6037.0,6037.0,6037.0,6059.0,6111.0


In [10]:
# let's also add a value range to this describe 

summary_stats["range"] = summary_stats["max"] - summary_stats["min"]
summary_stats

,count,mean,std,min,25%,50%,75%,max,range
bedroom_count,2152852.0,3.0,1.0,0.0,3.0,3.0,4.0,25.0,25.0
bath_count,2152852.0,2.0,1.0,0.0,2.0,2.0,3.0,32.0,32.0
finished_sq_feet,2144379.0,1863.0,1222.0,1.0,1257.0,1623.0,2208.0,952576.0,952575.0
tax_dollar_value,2152370.0,461896.0,699676.0,1.0,188170.0,327671.0,534527.0,98428909.0,98428908.0
year_built,2143526.0,1961.0,22.0,1801.0,1949.0,1958.0,1976.0,2016.0,215.0
taxamount,2148421.0,5635.0,8179.0,2.0,2535.0,4109.0,6414.0,1337756.0,1337754.0
fips,2152863.0,6048.0,20.0,6037.0,6037.0,6037.0,6059.0,6111.0,74.0


**``initial data familiarization/summary stats takeways: ``**

- the following features can be converted to "int" type columns/values: bedroom_count, year_built, and "fips"
- could "year_built" be better represented as a "datetime" type column/values ??

In [11]:
# continuing to explore the zillow dataframe: checking null values and percentage of column 
# here we see all columns contain "Null" values

zillow_df.isnull().any()

bedroom_count        True
bath_count           True
finished_sq_feet     True
tax_dollar_value     True
year_built           True
taxamount            True
fips                False
dtype: bool

In [12]:
# total number of missing values per column 

zillow_df.isnull().sum()

bedroom_count         11
bath_count            11
finished_sq_feet    8484
tax_dollar_value     493
year_built          9337
taxamount           4442
fips                   0
dtype: int64

In [13]:
# what percentage of Null values makeup ea. column 

round(zillow_df.isnull().mean(), 3)

bedroom_count       0.000
bath_count          0.000
finished_sq_feet    0.004
tax_dollar_value    0.000
year_built          0.004
taxamount           0.002
fips                0.000
dtype: float64

----
**``Dealing with Null Values in Zillow Dataset:``**

Each feature contains Null values:
- since missing/null values in ea. column represent <= ~1% of total observations, i am choosing to drop these rows/obversations
- reason being is that they may not be statistically significant in our overall analysis and we will still have a large enough sample/dataset to make inferences on future home value predictions

In [14]:
# let's handle the missing values and then the "column types"

zillow_df = zillow_df.dropna()
zillow_df.shape

(2140235, 7)

In [15]:
# difference in initial dataframe vs. dropped rows dataframe
# where ~12K rows/obversations have been dropped

print(f'initial dataset shape: {initial_shape}')
print(f'updated dataset with dropped rows: {zillow_df.shape}')

initial dataset shape: (2152863, 7)
updated dataset with dropped rows: (2140235, 7)


In [16]:
# converting the columns to correct type
# the following features can be converted to "int" type columns/values: bedroom_count, finished_sq_feet, and "fips"
# additionally, "year_built" can be converted to either "int" or "datetime" type


zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   bedroom_count     float64
 1   bath_count        float64
 2   finished_sq_feet  float64
 3   tax_dollar_value  float64
 4   year_built        float64
 5   taxamount         float64
 6   fips              float64
dtypes: float64(7)
memory usage: 130.6 MB


In [17]:
# using a for loop to print unique values per column
# value lists are abbreviated since there are too many unique values


for column in zillow_df.columns:
    
    print(f"Unique {column}'s count: {zillow_df[column].nunique()}")
    print(f'{zillow_df[column].unique()}', "\n")

Unique bedroom_count's count: 19
[ 4.  3.  0.  5.  2.  1.  6.  7.  8.  9. 25. 10. 11. 14. 15. 13. 12. 16.
 18.] 

Unique bath_count's count: 38
[ 2.    4.    0.    1.    2.5   3.5   3.    5.5   4.5   5.    1.5   7.5
  6.   10.    8.    7.   11.    9.    8.5   6.5  13.    9.5  20.   19.5
 12.   10.5  15.   16.    1.75 14.   18.    0.5  17.   12.5  11.5  14.5
 32.   19.  ] 

Unique finished_sq_feet's count: 10557
[ 3633.  1620.  2077. ...  8545.  7342. 24920.] 

Unique tax_dollar_value's count: 588178
[296425. 847770. 646760. ... 773358. 960756. 536061.] 

Unique year_built's count: 153
[2005. 2011. 1926. 1972. 1973. 1960. 1950. 1951. 2016. 2013. 2014. 2015.
 2012. 1944. 1953. 1984. 1971. 1989. 1932. 1978. 1961. 1967. 1981. 1966.
 1982. 1929. 1995. 1979. 1957. 1958. 1947. 1954. 1962. 1956. 1904. 1963.
 1940. 1980. 1987. 1964. 1935. 1928. 1955. 1948. 1959. 1965. 1974. 1952.
 1988. 1977. 1924. 1925. 1930. 1927. 1941. 1939. 1999. 1976. 1986. 1998.
 1968. 1975. 1991. 1994. 1936. 1993. 1890. 

In [18]:
# let's handle the needed "int" type columns first

zillow_df[["bedroom_count", "year_built", "fips"]] = zillow_df[["bedroom_count", "year_built", "fips"]].astype("int")
zillow_df.info() # checks out!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   bedroom_count     int64  
 1   bath_count        float64
 2   finished_sq_feet  float64
 3   tax_dollar_value  float64
 4   year_built        int64  
 5   taxamount         float64
 6   fips              int64  
dtypes: float64(4), int64(3)
memory usage: 130.6 MB


In [19]:
zillow_df.year_built.head(20)

4     2005
6     2011
7     1926
11    1972
14    1973
15    1960
18    1950
19    1950
20    1951
21    2016
23    2013
24    2014
25    2014
26    2014
27    2015
28    2013
29    2014
30    2013
31    2014
32    2015
Name: year_built, dtype: int64

In [20]:
# printing value_counts per individual column
for column in zillow_df.columns:
    print(column)
    print(zillow_df[column].value_counts())
    print("-----------------")
    print()

bedroom_count
3     962944
4     633608
2     334221
5     150671
6      25117
1      22895
7       4792
0       4397
8       1103
9        290
10       118
11        34
13        15
12        12
14         7
15         5
18         3
16         2
25         1
Name: bedroom_count, dtype: int64
-----------------

bath_count
2.00     942463
3.00     422398
1.00     412582
2.50     142827
4.00      82039
1.50      31157
3.50      28464
5.00      28306
4.50      19474
6.00      10717
5.50       6201
7.00       4381
0.00       4274
8.00       1681
6.50       1330
9.00        707
7.50        382
10.00       322
11.00       145
8.50        108
12.00        73
9.50         50
13.00        39
14.00        25
15.00        17
0.50         16
10.50        14
16.00        12
18.00         8
20.00         6
17.00         4
1.75          3
12.50         3
11.50         3
19.50         1
14.50         1
32.00         1
19.00         1
Name: bath_count, dtype: int64
-----------------

finished_sq_feet


----

### **``Data acquisition and preparation``**

- From ``"wrangle.py"`` file, I can now acquire and clean the zillow dataset with two (2) created functions:

1. **``zillow_df = get_zillow_data()``**

2. **``zillow_df = clean_zillow_data(zillow_df)``**